In [1]:
%cd ../../
%pwd

/home/hamed/code/Osprey


'/home/hamed/code/Osprey'

In [2]:
import torch
import pandas as pd

from transformers import AutoTokenizer, BertTokenizer, \
    DataCollatorForLanguageModeling, AutoModelForMaskedLM, \
    TrainingArguments, Trainer

from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer

/home/hamed/mambaforge/envs/osprey-cuda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "distilroberta-base"

In [4]:
dataset_path = "data/dataset-v2/train.csv"
df = pd.read_csv(dataset_path)
df["text"].fillna("", inplace=True)

df.head()

,Unnamed: 0,conv_id,msg_line,author_id,time,msg_char_count,msg_word_count,conv_size,nauthor,text,tagged_predator,predatory_conv
0,0,e621da5de598c9321a1d505ea95e6a2d,1,97964e7a9e8eb9cf78f2e4d7b2ff34c7,3.2,5,1,62,2,Hola.,0.0,0.0
1,1,e621da5de598c9321a1d505ea95e6a2d,2,0158d0d6781fc4d493f243d4caa49747,3.2,3,1,62,2,hi.,0.0,0.0
2,2,e621da5de598c9321a1d505ea95e6a2d,3,0158d0d6781fc4d493f243d4caa49747,3.2,9,2,62,2,whats up?,0.0,0.0
3,3,e621da5de598c9321a1d505ea95e6a2d,4,97964e7a9e8eb9cf78f2e4d7b2ff34c7,3.2,10,3,62,2,not a ton.,0.0,0.0
4,4,e621da5de598c9321a1d505ea95e6a2d,5,97964e7a9e8eb9cf78f2e4d7b2ff34c7,3.2,4,1,62,2,you?,0.0,0.0


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_set = [tokenizer(txt) for txt in df["text"].to_list()]

Token indices sequence length is longer than the specified maximum sequence length for this model (1472 > 512). Running this sequence through the model will result in indexing errors


In [6]:
# texts[0]
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [7]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
training_args = TrainingArguments(
    output_dir="output-pretraining",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=texts,
    # eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [12]:
trainer.train()